In [ ]:
'''Re-implementation in fenicsx, with some added features (half hair, hair at angle)'''

%reset -f

import sys
from lib_fem import *
from lib_plot import *

'''Pre-init'''

#empty cache
os.system('rm -rf /root/.cache/fenics/')

'''Scalings'''
# Length scale: mm
# Time scale: hours
# The numerical code is entirely dimensional.

'''Parameters'''

# Length scales (all in mm)
Er = 1.81  # embryo radius
r = .92 # margin radius
d = .12 # margin thickness
offset = .106 #initial downward displacement of EP in mm
l_diff = 0.1583 # diffusion length

# Time scales
tau = 0.5 # myosin regulation time scale [hrs]
emax = 1/3 # maximal contraction rate [1/hrs]
ve_time = 0.5 # margin viscoelastic time scale [hrs]

# Myosin scale
c0 = 1 #set to 1 (unknown units).

# Tension scale
Ts = 1 #set to 1 (unknown number of Newtons per unit myosin)

# Dimensionless biological parameters (controlling myosin regulation in response to stretching)
alpha = -1.0 # regulation offset
beta = 16.0 # sensitivity of myosin to stretching
zeta = 1/3 # amplitude of myosin variation
lam  = 4.0 # sensitivity of walkers to tension

# Tissue viscosity scale
mu = 0.1 # 2D tissue viscosity ( in units of [c0*Ts] * hours / mm )

# Hair parameters
fr = 500 #hair friction coefficient
angle = 0 #angle from horizontal in radians
xmin = 0 #minx for hair

####

# Margin visco-elasticity (emerging from walking kernel model, for information only)
E = 1/(lam*emax*ve_time) # = 1.5, margin elasticity ( in units of [c0*Ts] )
nu = 1/(lam*emax) # = 0.75, 1D margin viscosity ( in units of [c0*Ts] * hours )

# Dimensionless quantities are only calculated for theoretical interpretation, not used in calculation


'''Initial conditions and numerical settings'''

# Initialisation parameters
Camp = c0*zeta #initial myosin amplitude (perturbation from equilibrium)
initialT = c0*Ts #c0*Ts #initial tension (uniform along margin)

# Regulation settings
dt = 0.025 #0.05 #time step for FEM and mesh advection
subd = 10 #factor finer time step for margin timestepping
nrep = 321 #number of FEM time steps
remeshinterval = 20 #remesh after this many FEM timesteps

# general parameters
t0 = 0 #start time
N = 128  #number of margin segments, choose even
dataDir = 'test_output' # output data directory

# numerical parameters
meshres = .1 #mesh resolution in mm
pen = 2000 #penalty for weak enforcement of no slip boundary condition if divergence is non-zero, choose large
THorder = 1 #order of the FE method (choose at least 1. Velocity elements have degree THorder + 1)

# switches
regulate = 1 # if 0 tension has fixed gaussian profile WARNING regulate=0 has not yet been tested
remesh = 1 #remesh WARNING REMESHING RESETS THE LOCUS OF THE HAIR (IF PRESENT)
hair = 0 #hair friction on/off.
echo = 0
output = 1 #flag for saving output. 1 = on, 0 = off


'''Wrappers'''
# wrap parameters for easy query. All of these are immutable
#
general_parameters = [N,t0]
regulation_parameters = [alpha, beta, zeta, lam, emax, tau, l_diff, ve_time, c0, Ts]
initialisation_parameters = [c0,Camp,initialT, zeta, alpha]
tissue_parameters = [mu,fr*hair]
geometry_parameters = [Er,r,d,offset]
#
regulation_settings = [dt,subd,nrep,remeshinterval*remesh]    
fem_settings = [meshres,pen,THorder]
#
flags = [regulate,remesh,hair,echo,output]
    
"""Main program"""

'''Initialisation'''

t = t0 #current times is the initial time
currentradius = Er #current radius is the initial radius
imagestack = [] #empty imagestack for plot output
fig = None
ax = None

#prepare output
if output:
    dataout = prepare_output(dataDir)
    save_settings(dataDir,general_parameters,regulation_parameters,initialisation_parameters,tissue_parameters,geometry_parameters,regulation_settings,fem_settings,flags)
    
#initialise margin
margin_geom, ps, l, lt, cumt, th, ta, c, T, interpolators = initialise_mso(N,t,initialisation_parameters,geometry_parameters,regulate)

#mesh embryo
mesh, meshtags, markers = mso_meshing(N,meshres,currentradius,ps) #EP,EE,hair have indices 1,2,3 respectively

#Identify mesh regions with area growth
Ms, coefs = epiboly_init(mesh,geometry_parameters,THorder,t)

#Create Hair object
fricobj = get_fricobj(xmin,d,angle)

#Initialise solver (important to do this outside of the loop)
solver = create_solver()

In [ ]:
'''Loop'''
for j in range(nrep):
    
    print(j)
    
    '''Solve FEM'''
    
    #Calculate active stress object
    stressobj = get_stressobj_reg(margin_geom,interpolators,N,l,cumt,d, None)
    
    #obtain velocity and pressure fields
    u, p, W, un = mso_femsolve(N,mesh,markers,Ms,coefs,stressobj,fricobj,tissue_parameters,fem_settings,solver)
    
    '''Process solution'''
    
    #Evaluate u at margin points (using redefined margin)
    u_values = vel_on_margin(u,ps,mesh,N)
    
    #Calculate extension rate
    margvel, gd = calc_exrate(u_values,ta,l,N)
    
    '''Save and display output'''
    
    #save output
    if output:
        save_fem(mesh,str(j),dataDir,u,p)
        dataout = save_marg(dataout,t,T,c,gd,l,ps,margvel)
    
    #display information
    if echo:
        report_status(t,un,margvel,gd,T,c)
     
    #Plot Myosin, tension and contraction rate profiles
    fig, ax = plot_margin(N,t,cumt,c,T,gd,margvel,fig,ax,figureSize=8) 
    im = fig_to_array(fig) #Save in buffer to later collect in tif stack
    imagestack.append(im)    
    
    #Plot mesh
    #plot_mesh_pyvista(W,mesh)    
    
    if j % 20 == 0:
        #plot velocity
        plot_flow_pyvista(mesh,u,W.sub(0).collapse()[0],3,not j,backend='static')
        pass
    
    '''Time step margin'''
    
    #time-step margin coordinates, myosin and tension
    ps, c, T = regulate_margin(N,ps,u_values,gd,c,lt,T,Ms,coefs,mesh,regulation_parameters,regulation_settings,dt)
    
    #update quantities derived from new margin position
    margin_geom, l, lt, cumt, th, ta = updatelengths(ps,N)
    interpolators = interp_init(cumt,c,T,th)
    
    '''Time step mesh (remesh if desired)'''
    
    #move mesh and update radius
    mesh = move_mesh(mesh,u,dt)
    #Update radius
    currentradius+=dt*un
    
    #remesh if desired
    if remesh and ((j+1) % remeshinterval == 0):
        #Redefine margin
        margin_geom, ps, l, lt, cumt, th, ta, c, T, interpolators = redef_margin(N,margin_geom,l,cumt,interpolators)
        #Redefine mesh
        currentradius = det_newradius(mesh,currentradius)#,trim=5e-3) #trim currentradius to fit new mesh into old mesh
        newmesh, meshtags, markers = mso_meshing(N,meshres,currentradius,ps)
        #Interpolate Epiboly
        Ms = epiboly_remesh(newmesh,mesh,Ms,THorder)
        mesh = newmesh
   
    #update time step
    t += dt

    #Update contraction coefficients
    coefs = update_coefs(mesh,t)

#Save output
if output:
    save_output(dataDir,dataout,'Margin_data')
    save_imagestack(dataDir,imagestack,'Margin')